### While the shop hop is organized by regions, I want to know the most efficient way to complete as many shops as possible.  Time to cluster.

In [1]:
from config import maps_key, home_addr, alt_start_addr
from shared import *
import numpy as np
import pandas as pd
import requests
from sklearn import cluster

In [3]:
shops = pd.read_csv("input_data/geocoded_shops.csv")
shops

,region,shop name,address,lat,lng
0,metro,Affinity for Quilts,"2199 4th St, White Bear Lake, MN 55110",45.085587,-93.007865
1,metro,Country Loft Quilt & Design,"15161 Feller St NE, Forest Lake, MN 55025",45.244495,-93.026036
2,metro,Millie P's Quilt Shop,"219 E Main St, Anoka, MN 55303",45.198178,-93.388456
3,metro,Quilt Cove,"1960 Cliff Lake Rd, Suite 134, Eagan, MN 55122",44.794211,-93.204791
4,metro,Quilter's Haven,"2930 146th St, Suite 108, Rosemount, MN 55068",44.737330,-93.125043
5,metro,Twin Cities Quilting,"550 County Rd D, New Brighton, MN 55112",45.036242,-93.194271
6,east central,Country Caboose Quilts,"108 South Main St, Wahkon, MN 56386",46.116462,-93.520985
7,east central,Gone to Pieces Quilt Shop,"70 S Main, Kimball, MN 55353",45.313570,-94.300175
8,east central,Gruber's Quilt Shop,"310 4th Ave NE, Waite Park, MN 56387",45.559247,-94.206881
9,east central,The Noble Quilter,"19570 Holt St NW, Elk River, MN 55330",45.326394,-93.563834


In [4]:
home_lat, home_lng = get_lat_lng(home_addr, maps_key)

alt_start_lat, alt_start_lng = get_lat_lng(alt_start_addr, maps_key)

In [5]:
coords = ";".join(shops[["lng", "lat"]].apply(lambda row: ','.join(row.values.astype(str)), axis=1))
coords

'-93.0078652,45.0855867;-93.02603640000001,45.24449480000001;-93.38845620000001,45.1981781;-93.204791,44.7942112;-93.1250426,44.7373301;-93.1942715,45.0362424;-93.5209848,46.11646210000001;-94.3001749,45.313569799999996;-94.2068809,45.55924659999999;-93.5638344,45.3263943;-92.8846361,46.124488299999996;-93.553919,45.211043;-90.4445296,47.724412799999996;-92.0323471,46.827637200000005;-92.867063,46.8785185;-91.2991293,47.2589152;-92.45949770000001,46.746570399999996;-94.5479583,47.7299346;-95.8547502,47.8399399;-97.0209789,47.9408461;-94.881437,47.471895;-93.4395956,48.572548499999996;-94.2273891,46.355866799999994;-94.58444509999998,47.1004722;-95.1376307,46.44070869999999;-94.3081574,46.5805559;-96.10419420000001,46.284485100000005;-95.3780526,45.88617429999999;-94.5283382,45.1269287;-95.538025,44.8114027;-94.9442539,45.3011018;-94.36927790000001,44.896660600000004;-94.7148875,45.3821829;-94.52703259999998,44.5330574;-93.5813483,44.438185600000004;-93.42802900000001,44.47931;-93.99810

In [6]:
def get_names(shops, region=None):
    names = []
    
    if region:
        s = shops[shops["region"] == region]
    else:
        s = shops

    for i, r in s.iterrows():
        names.append(r['shop name'])
    
    return names


In [7]:
place_names = get_names(shops)
place_names

['Affinity for Quilts',
 'Country Loft Quilt & Design',
 "Millie P's Quilt Shop",
 'Quilt Cove',
 "Quilter's Haven",
 'Twin Cities Quilting',
 'Country Caboose Quilts',
 'Gone to Pieces Quilt Shop',
 "Gruber's Quilt Shop",
 'The Noble Quilter',
 'Quarry Quilts & Yarns',
 'Quilted Treasures',
 '47 Degrees North Quilt Shop',
 'Hannah Johnson Fabrics',
 'Hingeley Road Quilt Shop',
 'Quilt Corner',
 'Quilted Dog Quilt Shop',
 'Anderson Fabrics Quilt Shop',
 'Oklee Quilting Supply',
 "Quilter's Jem, LLC",
 "Sadie Rae's Quilt Shop",
 'Up North Quilt Shop',
 'Colorz Quilt Shop',
 'Front Porch Quilt Shop',
 'Hometown Crafts & Fabrics',
 'Mother Originals Quilt Shop',
 "Quilter's Cottage",
 "Dawn's Quilt Shop",
 "DeAnn's Country Village Shoppe",
 "Heather's Book Nook & Sew Much More",
 'Paisley Threads',
 'Quilt Haven on Main  ',
 'Sweetwater Cotton shoppe',
 "Cabby's Corner Quilt & Fabric",
 "Quilter's Dream",
 'Quilting by the Hearth',
 'River City Quilts',
 'Sewing Seeds Quilt Company',
 'Sp

In [8]:
# THIS USES THE OSMR API
# END USER IS RESPONSIBLE FOR RESPONSIBLE AND COMPLIANT API USAGE

matrix = requests.get(f"http://router.project-osrm.org/table/v1/driving/{coords}")

In [9]:
matrix

<Response [200]>

In [10]:
res = matrix.json()
res

{'code': 'Ok',
 'durations': [[0,
   1188.9,
   1928.2,
   2224.2,
   2556.6,
   1130.4,
   7313.1,
   6170.1,
   5428.5,
   2949.2,
   5054.1,
   2638.8,
   16866.3,
   9720.1,
   10716.3,
   13287.8,
   8395.5,
   18919.5,
   22114.2,
   21490.5,
   17009.3,
   20143.4,
   9800.8,
   14224.4,
   11683.3,
   10990.2,
   12388.8,
   9540.4,
   7278.7,
   11314.2,
   8315.8,
   6407.7,
   7308.4,
   7979.8,
   5279.3,
   4429.1,
   7674.7,
   9006.7,
   9007.5,
   4577.2,
   8983.8,
   14376.5,
   10113.9,
   12061.8,
   12176.4,
   12516.5,
   9261.5,
   7698.5,
   8979.7,
   6501.2,
   5880.1,
   7148],
  [1243.9,
   0,
   1989,
   2913.3,
   3245.7,
   1473.7,
   6769.5,
   6235.7,
   5494.1,
   3010,
   4188.3,
   2704.4,
   16000.5,
   8854.3,
   9850.5,
   12422,
   7529.7,
   18980.3,
   22175,
   21556.1,
   17070.1,
   19277.6,
   9861.6,
   14285.2,
   11744.1,
   11051,
   12454.4,
   9606,
   7706.4,
   11853.4,
   8381.4,
   6752.9,
   7374,
   8519,
   5968.4,
   5118.2,
 

In [11]:
df = pd.DataFrame(res["durations"], columns=place_names, index=place_names)
df

,Affinity for Quilts,Country Loft Quilt & Design,Millie P's Quilt Shop,Quilt Cove,Quilter's Haven,Twin Cities Quilting,Country Caboose Quilts,Gone to Pieces Quilt Shop,Gruber's Quilt Shop,The Noble Quilter,...,Michele's Quilting & Sewing Center,Old Alley Quilt Shop,Prairie Quilting,Stitchin' It Up Quilt Shop,Bluffview Quilt Shop,Calico Hutch,The Humble Heart,Pine Needles Quilt & Sew,Rather Bee Quilting,What's Brewing Quilt Shop
Affinity for Quilts,0.0,1188.9,1928.2,2224.2,2556.6,1130.4,7313.1,6170.1,5428.5,2949.2,...,10113.9,12061.8,12176.4,12516.5,9261.5,7698.5,8979.7,6501.2,5880.1,7148.0
Country Loft Quilt & Design,1243.9,0.0,1989.0,2913.3,3245.7,1473.7,6769.5,6235.7,5494.1,3010.0,...,10803.0,12750.9,12820.7,13055.7,9961.4,8387.6,9668.8,7190.3,6580.0,7847.9
Millie P's Quilt Shop,1997.9,2056.9,0.0,3052.1,3528.8,1362.5,5614.3,4924.4,4157.7,1250.4,...,10578.2,11793.3,11643.3,11902.6,10244.5,8276.1,9557.3,7473.4,6863.1,8131.0
Quilt Cove,2282.0,2950.1,3078.1,0.0,898.0,1902.8,8261.9,6698.1,6088.1,3898.0,...,8172.1,10120.0,10248.7,10725.5,8733.0,5756.7,7037.9,5638.8,5351.6,6619.5
Quilter's Haven,2657.0,3325.1,3509.0,892.1,0.0,2313.0,8893.9,7353.9,6743.9,4530.0,...,8531.6,10479.5,10622.7,11099.5,7930.7,6116.2,7397.4,4836.5,4549.3,5817.2
Twin Cities Quilting,1150.7,1477.6,1363.4,1816.4,2313.7,0.0,6748.3,5591.5,4849.9,2384.4,...,9706.1,11654.0,11562.9,11819.0,9216.7,7290.7,8571.9,6445.6,5835.3,7103.2
Country Caboose Quilts,7386.8,7199.2,5588.8,8327.1,8917.7,6751.4,0.0,6503.2,5148.8,4664.0,...,15791.9,17007.0,16857.0,15646.8,15633.4,13403.7,14684.9,12862.3,12252.0,13519.9
Gone to Pieces Quilt Shop,6225.6,6320.3,4893.4,6755.3,7408.0,5548.5,6535.2,0.0,1468.1,4329.4,...,11019.5,10680.0,10530.0,9581.7,14415.3,11831.9,10821.5,11644.2,11033.9,12301.8
Gruber's Quilt Shop,5525.1,5619.8,4134.7,6190.0,6842.7,4848.0,5226.3,1494.9,0.0,3431.2,...,12514.4,12174.9,12024.9,10588.8,13714.8,11266.6,12316.4,10943.7,10333.4,11601.3
The Noble Quilter,3008.6,3067.6,1210.6,3948.9,4539.5,2373.2,4662.2,4277.8,3389.9,0.0,...,11413.7,12628.8,12478.8,12521.2,11255.2,9025.5,10306.7,8484.1,7873.8,9141.7


In [12]:
df.to_csv("input_data/distance_matrix.csv")

### Affinity Propogation

In [13]:
X = df.to_numpy()
X

array([[   0. , 1188.9, 1928.2, ..., 6501.2, 5880.1, 7148. ],
       [1243.9,    0. , 1989. , ..., 7190.3, 6580. , 7847.9],
       [1997.9, 2056.9,    0. , ..., 7473.4, 6863.1, 8131. ],
       ...,
       [6591.5, 7259.6, 7637.2, ...,    0. , 3262.9, 3757.4],
       [5863.1, 6561.5, 6939.1, ..., 3263.8,    0. , 1267.9],
       [7126. , 7824.4, 8202. , ..., 3784.2, 1262.9,    0. ]])

In [14]:
clustering = cluster.AffinityPropagation(random_state=7).fit(X)
clustering

AffinityPropagation(random_state=7)

In [15]:
clustering.labels_

array([0, 0, 0, 0, 0, 0, 3, 4, 4, 0, 1, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2,
       3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 0, 0, 6, 5, 5, 0, 5, 5, 5, 5,
       5, 5, 6, 6, 6, 6, 6, 6])

In [16]:
new_groups_df = shops.copy()

In [17]:
new_groups_df["affinity_cluster"] = clustering.labels_
new_groups_df

,region,shop name,address,lat,lng,affinity_cluster
0,metro,Affinity for Quilts,"2199 4th St, White Bear Lake, MN 55110",45.085587,-93.007865,0
1,metro,Country Loft Quilt & Design,"15161 Feller St NE, Forest Lake, MN 55025",45.244495,-93.026036,0
2,metro,Millie P's Quilt Shop,"219 E Main St, Anoka, MN 55303",45.198178,-93.388456,0
3,metro,Quilt Cove,"1960 Cliff Lake Rd, Suite 134, Eagan, MN 55122",44.794211,-93.204791,0
4,metro,Quilter's Haven,"2930 146th St, Suite 108, Rosemount, MN 55068",44.737330,-93.125043,0
5,metro,Twin Cities Quilting,"550 County Rd D, New Brighton, MN 55112",45.036242,-93.194271,0
6,east central,Country Caboose Quilts,"108 South Main St, Wahkon, MN 56386",46.116462,-93.520985,3
7,east central,Gone to Pieces Quilt Shop,"70 S Main, Kimball, MN 55353",45.313570,-94.300175,4
8,east central,Gruber's Quilt Shop,"310 4th Ave NE, Waite Park, MN 56387",45.559247,-94.206881,4
9,east central,The Noble Quilter,"19570 Holt St NW, Elk River, MN 55330",45.326394,-93.563834,0


In [18]:
new_groups_df.to_csv("input_data/affinity_clusters.csv", index=False)

### Mapping new clusters

In [20]:
shops = pd.read_csv("input_data/affinity_clusters.csv")
shops

,region,shop name,address,lat,lng,affinity_cluster
0,metro,Affinity for Quilts,"2199 4th St, White Bear Lake, MN 55110",45.085587,-93.007865,0
1,metro,Country Loft Quilt & Design,"15161 Feller St NE, Forest Lake, MN 55025",45.244495,-93.026036,0
2,metro,Millie P's Quilt Shop,"219 E Main St, Anoka, MN 55303",45.198178,-93.388456,0
3,metro,Quilt Cove,"1960 Cliff Lake Rd, Suite 134, Eagan, MN 55122",44.794211,-93.204791,0
4,metro,Quilter's Haven,"2930 146th St, Suite 108, Rosemount, MN 55068",44.737330,-93.125043,0
5,metro,Twin Cities Quilting,"550 County Rd D, New Brighton, MN 55112",45.036242,-93.194271,0
6,east central,Country Caboose Quilts,"108 South Main St, Wahkon, MN 56386",46.116462,-93.520985,3
7,east central,Gone to Pieces Quilt Shop,"70 S Main, Kimball, MN 55353",45.313570,-94.300175,4
8,east central,Gruber's Quilt Shop,"310 4th Ave NE, Waite Park, MN 56387",45.559247,-94.206881,4
9,east central,The Noble Quilter,"19570 Holt St NW, Elk River, MN 55330",45.326394,-93.563834,0


In [24]:
# THIS USES THE OSMR API
# END USER IS RESPONSIBLE FOR RESPONSIBLE AND COMPLIANT API USAGE

m, distances, durations = map_all_regions(shops, home_lat, home_lng, region_col="affinity_cluster")

In [25]:
m

In [26]:
distances

{0: 223.3656654280637,
 3: 473.2354878397356,
 4: 356.9594367877515,
 1: 566.1734002758895,
 2: 842.9370425143227,
 6: 388.1510432848249,
 5: 420.2987560117812}

In [27]:
durations

{0: 5.305583333333333,
 3: 10.357194444444445,
 4: 7.962083333333334,
 1: 11.206416666666666,
 2: 16.91375,
 6: 8.313555555555556,
 5: 9.143305555555555}

### Hmm, this is nice, but I want to know which regions the shops belong to on these routes.

In [28]:
rm = add_all_region_markers(shops, existing_map=m)

In [29]:
rm

In [31]:
m.save("output_data/index.html")